# PDF Parsing 

Team Name: KANDY

Date: November 06, 2022

Project: Analyzing Information on Car Collision in Canada

Competition: HACKED(beta) 2022 at the University of Alberta

Author: Jamie Lee

Team Members: Min Joh, Jamie Lee, Taekwan Yoon, Yongbin Kim, Dohyun Kim

Description of file:

This Jupyter Notebook parse table data from pdf file and merge all the csv files after. 

## 1. Setups

### 1.1 Installing Modules

In [1]:
pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### 1.2 Importing Modules and Read File

In [25]:
from PyPDF2 import PdfFileReader
from glob import glob
from google.colab import drive
import pandas as pd
import os
import csv

drive.mount('/content/drive')
file_path = '/content/drive/Shareddrives/KANDY-hacked-beta/Data_Sets/alberta_collision_statistics/trans-alberta-traffic-collision-statistics-2018.pdf'
reader = PdfFileReader(file_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 2. PDF Parsing

### 2.1 Find Page Number

In [26]:
def find_page_num():
  title = "Collision occurrence by month"
  for i in range(30):
    page_obj = reader.getPage(i)
    if i > 15:
      for j in range(10):
        try:
          page_title = page_obj.extractText().split('\n')[j]
        except IndexError:
            page_title = ""
        if title.lower() in page_title.lower():
          return i

find_page_num()

20

### 2.2 Get All Context from Given Page Number

In [27]:
def get_context(page_num):
  # extract all the contexts from given page_num
  page_obj = reader.getPage(page_num)
  context_list = [page_obj.extractText().split('\n')]
  context_list  = [val for sublist in context_list for val in sublist]

  # taking monthly data and remove other unwated contexts
  index = 0 
  for i in context_list:
    if i.startswith("January"):
      context_list = context_list[index:]
    index += 1
  index = 0 
  for i in context_list:
    if i.endswith("Total Number"):
      context_list = context_list[:index]
    index += 1
  
  # remove unwanted string from last item in list
  last_list = context_list[len(context_list)-1].split('U')
  last_list = last_list[:1]
  context_list = context_list[:-1] + last_list

  monthly_context = ''.join(context_list).split()
  monthly_context_new = []
  for string in monthly_context:
    character = list(string)
    for idx in range(len(character)):
      if character[idx].isalpha():
        character[idx] = ""
    character = ''.join(character)
    monthly_context_new.append(character)
    monthly_context = monthly_context_new[1:]
  return monthly_context

page_num = find_page_num()
get_context(page_num)

['15',
 '6.1',
 '1,246',
 '9.7',
 '13,361',
 '10.3',
 '14,622',
 '10.3',
 '15',
 '6.1',
 '1,156',
 '9.0',
 '13,396',
 '10.3',
 '14,567',
 '10.2',
 '11',
 '4.5',
 '1,154',
 '9.0',
 '11,458',
 '8.8',
 '12,623',
 '8.9',
 '15',
 '6.1',
 '803',
 '6.2',
 '9,268',
 '7.2',
 '10,086',
 '7.1',
 '18',
 '7.3',
 '996',
 '7.7',
 '9,134',
 '7.1',
 '10,148',
 '7.1',
 '27',
 '11.0',
 '1,045',
 '8.1',
 '9,728',
 '7.5',
 '10,800',
 '7.6',
 '26',
 '10.6',
 '1,026',
 '8.0',
 '9,248',
 '7.1',
 '10,300',
 '7.2',
 '27',
 '11.0',
 '978',
 '7.6',
 '8,716',
 '6.7',
 '9,721',
 '6.8',
 '23',
 '9.3',
 '1,040',
 '8.1',
 '9,224',
 '7.1',
 '10,287',
 '7.2',
 '27',
 '11.0',
 '1,139',
 '8.9',
 '11,163',
 '8.6',
 '12,329',
 '8.6',
 '21',
 '8.5',
 '1,200',
 '9.3',
 '13,707',
 '10.6',
 '14,928',
 '10.5',
 '21',
 '8.5',
 '1,069',
 '8.3',
 '11,088',
 '8.6',
 '12,178',
 '8.5']

### 2.3 Create csv File and Initialize

In [31]:
def create_file(context_list):
  fieldnames = ['Year', 'Month', 'Fatal Collisions', 'Non-Fatal Injury Collisions', 'Property Damage Collisions', 'Total Collsions']

  with open('/content/drive/Shareddrives/KANDY-hacked-beta/Data_Sets/alberta_collision_statistics_csv/trans-alberta-traffic-collision-statistics-2018.csv', 'w') as file:
    writer = csv.writer(file)
    writer.writerow(fieldnames)

    row_list = []
    for index in range(100):
      if index % 8 == 0:
        row_list.append(file_path[len(file_path) - 8:-4])
        if index == 0:
          row_list.append('January')
        elif index == 8:
          row_list.append('Feburary')
        elif index == 16:
          row_list.append('March')
        elif index == 24:
          row_list.append('April')
        elif index == 32:
          row_list.append('May')
        elif index == 40:
          row_list.append('June')
        elif index == 48:
          row_list.append('July')
        elif index == 56:
          row_list.append('August')
        elif index == 64:
          row_list.append('September')
        elif index == 72:
          row_list.append('October')
        elif index == 80:
          row_list.append('November')
        elif index == 88:
          row_list.append('December')
        elif index == 96:
          break
        row_list.append(context_list[index])
      elif index % 8 == 2:
        row_list.append(context_list[index])
      elif index % 8 == 4:
        row_list.append(context_list[index])
      elif index % 8 == 6:
        row_list.append(context_list[index])
      elif index % 8 == 7:
        print(row_list)
        writer.writerow(row_list)
        row_list = []

page_num = find_page_num()
context_file = get_context(page_num)
create_file(context_file)

['2018', 'January', '15', '1,246', '13,361', '14,622']
['2018', 'Feburary', '15', '1,156', '13,396', '14,567']
['2018', 'March', '11', '1,154', '11,458', '12,623']
['2018', 'April', '15', '803', '9,268', '10,086']
['2018', 'May', '18', '996', '9,134', '10,148']
['2018', 'June', '27', '1,045', '9,728', '10,800']
['2018', 'July', '26', '1,026', '9,248', '10,300']
['2018', 'August', '27', '978', '8,716', '9,721']
['2018', 'September', '23', '1,040', '9,224', '10,287']
['2018', 'October', '27', '1,139', '11,163', '12,329']
['2018', 'November', '21', '1,200', '13,707', '14,928']
['2018', 'December', '21', '1,069', '11,088', '12,178']


### 3. Merge all csv Files into Single File

In [32]:
import glob

path = "/content/drive/Shareddrives/KANDY-hacked-beta/Data_Sets/alberta_collision_statistics_csv/"

file_list = glob.glob(path + "/*.csv")
print('File names:', file_list)
pd.concat(map(pd.read_csv, file_list), ignore_index=True).sort_values(by='Year').to_csv("/content/drive/Shareddrives/KANDY-hacked-beta/Data_Sets/alberta_collision_statistics_csv/alberta_collision_statistics.csv",index=False)

File names: ['/content/drive/Shareddrives/KANDY-hacked-beta/Data_Sets/alberta_collision_statistics_csv/trans-alberta-traffic-collision-statistics-2019.csv', '/content/drive/Shareddrives/KANDY-hacked-beta/Data_Sets/alberta_collision_statistics_csv/trans-alberta-traffic-collision-statistics-1998.csv', '/content/drive/Shareddrives/KANDY-hacked-beta/Data_Sets/alberta_collision_statistics_csv/trans-alberta-traffic-collision-statistics-1999.csv', '/content/drive/Shareddrives/KANDY-hacked-beta/Data_Sets/alberta_collision_statistics_csv/trans-alberta-traffic-collision-statistics-2000.csv', '/content/drive/Shareddrives/KANDY-hacked-beta/Data_Sets/alberta_collision_statistics_csv/trans-alberta-traffic-collision-statistics-2001.csv', '/content/drive/Shareddrives/KANDY-hacked-beta/Data_Sets/alberta_collision_statistics_csv/trans-alberta-traffic-collision-statistics-2002.csv', '/content/drive/Shareddrives/KANDY-hacked-beta/Data_Sets/alberta_collision_statistics_csv/trans-alberta-traffic-collision-s